First Generate The graph. 

*   200 Nodes
*   Average connectivity of 2 nodes
*   Distance is 1/20 of mile between nodes
*   Bidirectional

In [22]:
import networkx as nx
seed=1000                                           # seed the graph for reproducibility, you should be doing this  

def generateGraph(N, connect):
  
  prob = connect / (N-1)
  

  G = nx.gnp_random_graph (N, prob, seed=seed )       # here we create a random binomial graph with 20 nodes and an average (expected) connectivity of 10*.3= 3.

  
  while True:
    edges = nx.bfs_edges(G, 0)
    nodes = [0] + [v for u, v in edges]

    unreachable = [x for x in range(N) if x not in nodes]

    if not unreachable:
      break

    G.add_edge(random.randrange(0, N, 1), unreachable[0])
    

  print("Edges:")
  print(G.edges())

  return G


Function for generating a journey where the length is at least 2

In [23]:
import random
random.seed(seed)
def randomJourney(G, N):
  start = 0
  stop = 0

  while True:
    start = random.randrange(0, N, 1)
    stop = random.randrange(0, N, 1)

    if nx.shortest_path_length(G, start, stop) >= 2:
      break

  return start, stop

Create customer class
* Pick up node
* Drop off node

In [24]:
class Customer:
  
  def __init__(self, id, G, N):
    self.id = id
    self.start, self.stop = randomJourney(G, N)

  def __repr__(self):
    return str(self.id)

Create vehicle class


*   Up to 5 customers
*   Labeled 1-N
* 30 mph




In [25]:
class Vehicle:
  def __init__(self, number, currentNode):
    self.number = number
    self.currentNode =  currentNode
    self.path = []
    self.passengers = []
    self.assigned = []
    self.newPickup = False
    self.mileage = 0

  def generatePickUpPath(self, G, startNode, assList):
    if not assList:
      return [];

    bestPath = [];
    bestLength = float('inf');
    for p in assList:
      newList = [];
      #making a newlist that doesn't have the current passenger
      for a in assList:
        if a != p:
          newList.append(a);

      #generating the shortest path
      currPath = nx.shortest_path(G, self.currentNode, p.start, None) + self.generatePickUpPath(G, p.start, newList);
      #count the path length
      currLength = len(currPath);

      #compare with the best so far
      if currLength < bestLength:
        bestPath = currPath;
        bestLength = currLength;

    return bestPath;
    
  def generateDropOffPath(self, G, startNode, passList):
    if not passList:
      return [];

    bestPath = [];
    bestLength = float('inf');
    for p in passList:
      newList = [];
      #making a newlist that doesn't have the current passenger
      for a in passList:
        if a != p:
          newList.append(a);

      #generating the shortest path
      currPath = nx.shortest_path(G, self.currentNode, p.stop, None) + self.generateDropOffPath(G, p.stop, newList);
      #count the path length
      currLength = len(currPath);

      #compare with the best so far
      if currLength < bestLength:
        bestPath = currPath;
        bestLength = currLength;

    return bestPath;

  def followPath(self):
    if not self.path:
      return;

    self.currentNode = self.path.pop();
    self.mileage = self.mileage + (1/20)
    delivered = []; 
    picked = [];

    for a in self.assigned:
      if a.start == self.currentNode:
        picked.append(a);

    for a in picked:
      self.assigned.remove(a);
      self.passengers.append(a);

    for p in self.passengers:
      if p.stop == self.currentNode:
        delivered.append(p);

    for p in delivered:
      self.passengers.remove(p);

    return;


Generate cars
* 20 vehicles
* all start on node 0 (\***ASSUMPTION***)
* passenger list starts off empty
* no path at beginning. All cars are idle

Function for finding closest car

In [26]:
def closestVehicle(G, node, vehicles):
  closest = Vehicle(-1, 0)
  dist = 1000000
  
  for v in vehicles:
    d = nx.shortest_path_length(G, v.currentNode, node)
    if d < dist and (len(v.assigned) + len(v.passengers) < 5):
      closest = v
      dist = d

  return closest

Run simulation
* There are 600 timesteps to an hour, so one timestep is 6 seconds
* 100-150 reservations per hour, so 1 reservation ever 4-6 timesteps
* 8 hours, 4800 timesteps over simulation

In [27]:
def simulation(N, connect, cars):

  G = generateGraph(N, connect)

  vehicles = []

  for i in range(cars):
    vehicles.append(Vehicle(i + 1, 0))

  unassigned = []

  for i in range(4800):
    
    #Generate random passeners every 5 timesteps or 30 seconds
    #make more random this will do for now
    if i % random.randrange(4, 7, 1) == 0:
      unassigned.append(Customer(i, G, N))

    copy = list(unassigned)
    for cust in copy:
      #Find closest car and assign
      closest = closestVehicle(G, cust.start, vehicles)
      
      #if nothing was found
      if closest.number != -1:
        closest.newPickup = True
        closest.assigned.append(cust)
        unassigned.remove(cust)

    
    
    for car in vehicles:
      #print(car.number, car.assigned, car.passengers, car.path)

      if car.newPickup:
        car.path = car.generatePickUpPath(G, car.currentNode, car.assigned)
        car.newPickup = False;

      elif not car.path and car.passengers:
        car.path = car.generateDropOffPath(G, car.currentNode, car.passengers)
      
      #otherwise idle

      car.followPath()

  average = 0
  for car in vehicles:
    average += car.mileage
  print("Average Mileage:", average/len(vehicles))

In [29]:
print("200 node graph, average connectivity of 2, 30 vehicles")
simulation(200, 2, 30)
print()
print("200 node graph, average connectivity of 2, 60 vehicles")
simulation(200, 2, 60)
print()
print("200 node graph, average connectivity of 4, 30 vehicles")
simulation(200, 4, 30)

200 node graph, average connectivity of 2, 30 vehicles
Edges:
[(0, 123), (0, 142), (1, 183), (1, 100), (2, 38), (2, 73), (3, 57), (3, 112), (4, 125), (4, 144), (5, 159), (5, 175), (5, 96), (6, 172), (6, 128), (7, 97), (7, 127), (7, 156), (8, 140), (8, 190), (9, 48), (9, 95), (10, 119), (10, 180), (10, 188), (11, 33), (11, 66), (11, 135), (11, 139), (12, 41), (12, 110), (12, 156), (12, 174), (12, 25), (13, 70), (13, 117), (14, 137), (14, 170), (14, 171), (14, 182), (15, 48), (15, 138), (16, 32), (16, 75), (16, 81), (17, 100), (17, 139), (18, 173), (18, 190), (18, 193), (18, 199), (19, 32), (19, 33), (19, 68), (20, 72), (20, 81), (20, 150), (21, 93), (21, 141), (22, 61), (22, 186), (23, 105), (23, 120), (24, 90), (25, 108), (26, 93), (26, 128), (26, 141), (26, 181), (26, 153), (27, 86), (28, 94), (29, 45), (30, 65), (31, 128), (31, 154), (32, 86), (32, 189), (33, 123), (34, 50), (34, 106), (34, 120), (34, 131), (34, 37), (35, 141), (35, 162), (35, 174), (36, 134), (36, 156), (36, 155), (